<a href="https://colab.research.google.com/github/samibahig/IFT6135/blob/main/B_MNIST_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyTorch Tutorial

IFT6135 – Representation Learning

A Deep Learning Course, January 2020

By Chin-Wei Huang 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms


from_numpy = torch.from_numpy

In [ ]:
batch_size = 64
num_epochs = 10
cuda = torch.cuda.is_available()
if cuda:
    print('cuda is available')
else:
    print('cuda is not available')

store_every = 1000
lr0 = 0.02
#model_type = 'MLP'
model_type = 'CNN'

cuda is available


Loading data

In [ ]:
mnist_transforms = torchvision.transforms.Compose(
        [torchvision.transforms.ToTensor()])
mnist_train = torchvision.datasets.MNIST(
        root='./data', train=True, 
        transform=mnist_transforms, download=True)
mnist_test = torchvision.datasets.MNIST(
        root='./data', train=False, 
        transform=mnist_transforms, download=True)

train_loader = torch.utils.data.DataLoader(
        mnist_train, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(
        mnist_test, batch_size=batch_size, shuffle=True, num_workers=2)


  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 20640901.86it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


32768it [00:00, 331475.33it/s]
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 5370910.99it/s]                           
8192it [00:00, 125878.75it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


Defining customized modules

In [ ]:
class ResLinear(nn.Module):

    def __init__(self, in_features, out_features, activation=nn.ReLU()):
        super(ResLinear, self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.activation = activation
        
        self.linear = nn.Linear(in_features, out_features)
        if in_features != out_features:
            self.project_linear = nn.Linear(in_features, out_features)
        
    def forward(self, x):
        inner = self.activation(self.linear(x))
        if self.in_features != self.out_features:
            skip = self.project_linear(x)
        else:
            skip = x
        return inner + skip


class Flatten(nn.Module):
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x

Building model

In [ ]:
if model_type == 'MLP':        
    model = nn.Sequential(
        ResLinear(784, 312),
        nn.ReLU(),
        ResLinear(312, 312),
        nn.ReLU(),
        ResLinear(312, 10)
    )
elif model_type == 'CNN':
    model = nn.Sequential(
        nn.Conv2d(1, 16, 5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(16, 16, 5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        Flatten(),
        ResLinear(256, 100),
        nn.ReLU(),
        ResLinear(100, 10)
    )

if cuda:
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr0)

Learning rate schedule

In [ ]:
def adjust_lr(optimizer, epoch, total_epochs):
    lr = lr0 * (0.1 ** (epoch / float(total_epochs)))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
# `torch.optim.lr_scheduler` can be your good friend

Evaluation functions

In [ ]:
def accuracy(proba, y):
    correct = torch.eq(proba.max(1)[1], y).sum().type(torch.FloatTensor)
    return correct / y.size(0)
    
    
def evaluate(dataset_loader, criterion):
    LOSSES = 0
    COUNTER = 0
    for batch in dataset_loader:
        optimizer.zero_grad()

        x, y = batch
        if model_type == 'MLP':
            x = x.view(-1,784)
            y = y.view(-1)
        elif model_type == 'CNN':
            x = x.view(-1,1,28,28)
            y = y.view(-1)
        if cuda:
            x = x.cuda()
            y = y.cuda()
            
        loss = criterion(model(x), y)
        n = y.size(0)
        LOSSES += loss.sum().data.cpu().numpy() * n
        COUNTER += n
    return LOSSES / float(COUNTER)

Training loop

In [ ]:
LOSSES = 0
COUNTER = 0
ITERATIONS = 0
learning_curve_nll_train = list()
learning_curve_nll_test = list()
learning_curve_acc_train = list()
learning_curve_acc_test = list()
for e in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()

        x, y = batch
        if model_type == 'MLP':
            x = x.view(-1,784)
            y = y.view(-1)
        elif model_type == 'CNN':
            x = x.view(-1,1,28,28)
            y = y.view(-1)
        if cuda:
            x = x.cuda()
            y = y.cuda()
            
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()
        
        n = y.size(0)
        LOSSES += loss.sum().data.cpu().numpy() * n
        COUNTER += n
        ITERATIONS += 1
        if ITERATIONS%(store_every/5) == 0:
            avg_loss = LOSSES / float(COUNTER)
            LOSSES = 0
            COUNTER = 0
            print(" Iteration {}: TRAIN {}".format(
                ITERATIONS, avg_loss))
    
        if ITERATIONS%(store_every) == 0:     
            train_loss = evaluate(train_loader, criterion)
            learning_curve_nll_train.append(train_loss)
            test_loss = evaluate(test_loader, criterion)
            learning_curve_nll_test.append(test_loss)

            train_acc = evaluate(train_loader, accuracy)
            learning_curve_acc_train.append(train_acc)
            test_acc = evaluate(test_loader, accuracy)
            learning_curve_acc_test.append(test_acc)
                    
            print(" [NLL] TRAIN {} / TEST {}".format(
                train_loss, test_loss))
            print(" [ACC] TRAIN {} / TEST {}".format(
                train_acc, test_acc))
    
    adjust_lr(optimizer, e+1, num_epochs)
    

 Iteration 200: TRAIN 1.7470000556111336
 Iteration 400: TRAIN 0.4628930714726448
 Iteration 600: TRAIN 0.2989757400378585
 Iteration 800: TRAIN 0.23055737674236298


NameError: ignored